In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import torch

Explanation

Extract Unique Nodes: Get unique source and destination IP addresses.

Map Nodes to Indices: Create a mapping from node values to integer indices.

Map Source and Destination to Indices: Convert source and destination IPs to indices using the mapping.

Create Adjacency List: Store the adjacency list as a NumPy array.

Create Edge Index: Use the mapped indices to create the edge_index tensor for PyTorch Geometric

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install torch_sparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 417, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 320, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 194, in _build_one
    wheel_path = _build_one_inside_env(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/wheel_builder.py", line 241, in _build_one_inside_env
    wheel_path = build_wheel_lega

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/TON_IoT_Train_Test_Network.csv')

In [ ]:
df.type.value_counts()

type
normal        300000
scanning       20000
dos            20000
injection      20000
ddos           20000
password       20000
xss            20000
ransomware     20000
backdoor       20000
mitm            1043
Name: count, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461043 entries, 0 to 461042
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ts                      461043 non-null  int64  
 1   src_ip                  461043 non-null  object 
 2   src_port                461043 non-null  int64  
 3   dst_ip                  461043 non-null  object 
 4   dst_port                461043 non-null  int64  
 5   proto                   461043 non-null  object 
 6   service                 461043 non-null  object 
 7   duration                461043 non-null  float64
 8   src_bytes               461043 non-null  int64  
 9   dst_bytes               461043 non-null  int64  
 10  conn_state              461043 non-null  object 
 11  missed_bytes            461043 non-null  int64  
 12  src_pkts                461043 non-null  int64  
 13  src_ip_bytes            461043 non-null  int64  
 14  dst_pkts            

In [ ]:
df['type'].unique()

array(['normal', 'scanning', 'dos', 'injection', 'ddos', 'password',
       'xss', 'ransomware', 'backdoor', 'mitm'], dtype=object)

In [ ]:
df['weird_notice'].unique()

array(['F', '-'], dtype=object)

In [ ]:
df['weird_addl'].unique()

array(['-', '46', '48', '43'], dtype=object)

In [ ]:
df['http_resp_mime_types'].unique()

array(['-', 'application/ocsp-response', 'application/xml', 'text/json',
       'text/plain', 'application/x-debian-package',
       'application/vnd.ms-cab-compressed', 'image/png', 'image/jpeg',
       'text/html'], dtype=object)

In [ ]:
from sklearn import preprocessing
df_ = df.copy()
for i in range(5,len(df_.columns)):
    if df_.iloc[:,i].dtypes == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(df_.iloc[:,i])
        df_.iloc[:,i] = le.transform(df_.iloc[:,i])
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461043 entries, 0 to 461042
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ts                      461043 non-null  int64  
 1   src_ip                  461043 non-null  object 
 2   src_port                461043 non-null  int64  
 3   dst_ip                  461043 non-null  object 
 4   dst_port                461043 non-null  int64  
 5   proto                   461043 non-null  object 
 6   service                 461043 non-null  object 
 7   duration                461043 non-null  float64
 8   src_bytes               461043 non-null  int64  
 9   dst_bytes               461043 non-null  int64  
 10  conn_state              461043 non-null  object 
 11  missed_bytes            461043 non-null  int64  
 12  src_pkts                461043 non-null  int64  
 13  src_ip_bytes            461043 non-null  int64  
 14  dst_pkts            

In [ ]:
del df_['ts']

In [ ]:
df_.head()

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,3.122.49.24,1883,192.168.1.152,52976,1,0,80549.530260,1762852,41933215,0,...,0,0,0,0,0,5,0,1,0,5
1,192.168.1.79,47260,192.168.1.255,15600,2,0,0.000000,0,0,6,...,0,0,0,0,0,0,0,0,0,5
2,192.168.1.152,1880,192.168.1.152,51782,1,0,0.000000,0,0,0,...,0,0,0,0,0,5,0,1,0,5
3,192.168.1.152,34296,192.168.1.152,10502,1,0,0.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,192.168.1.152,46608,192.168.1.190,53,2,3,0.000549,0,298,12,...,0,0,0,0,0,6,0,1,0,5


In [ ]:

len(df_['dst_ip'].unique())

5268

This function selects random IPs from the srcip_list, as many rows as the df_ dataframe.

In [ ]:
srcip_list = ["172.16.0.1","172.17.0.1","172.18.0.1","172.19.0.1","172.20.0.1","172.21.0.1","172.22.0.1","172.23.0.1",
              "172.24.0.1","172.25.0.1","172.26.0.1","172.27.0.1","172.28.0.1","172.29.0.1","172.30.0.1","172.31.0.1"]
df_['src_ip'] = np.random.choice(srcip_list, df_.shape[0])

In [ ]:
df_['src_ip'] = df_['src_ip'].astype(str)+":"+df_['src_port'].astype(str)
df_['dst_ip'] = df_['dst_ip'].astype(str)+":"+df_['dst_port'].astype(str)
df_.drop(columns=['src_port','dst_port'], inplace=True)
df_.head()

,src_ip,dst_ip,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,172.16.0.1:1883,192.168.1.152:52976,1,0,80549.530260,1762852,41933215,0,0,252181,...,0,0,0,0,0,5,0,1,0,5
1,172.21.0.1:47260,192.168.1.255:15600,2,0,0.000000,0,0,6,0,1,...,0,0,0,0,0,0,0,0,0,5
2,172.24.0.1:1880,192.168.1.152:51782,1,0,0.000000,0,0,0,0,0,...,0,0,0,0,0,5,0,1,0,5
3,172.20.0.1:34296,192.168.1.152:10502,1,0,0.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,172.30.0.1:46608,192.168.1.190:53,2,3,0.000549,0,298,12,0,0,...,0,0,0,0,0,6,0,1,0,5


In [ ]:
df_.shape

(461043, 42)

The goal of this section is to create a list of unique nodes (IP addresses in this case) from the dataset, which will be used to construct the graph. Nodes in a graph represent entities, and edges represent relationships or interactions between these entities. In a network intrusion detection dataset, nodes typically represent network devices (identified by their IP addresses), and edges represent connections or communication between these devices

In [ ]:
df_.to_pickle("iot.pkl")

#Extract Unique Nodes
#Converts the series of source IP addresses into a set,
#automatically removing any duplicate IP addresses.
#This ensures that srcnode contains only unique source IP addresses.
srcnode = set(df_['src_ip'])
dtnode = set(df_['dst_ip'])

nodes = list(dtnode.union(srcnode))
np.save("nodes.npy", nodes)

In [ ]:
node_to_index = {node: i for i, node in enumerate(nodes)}

# Map src and dst to indices
src = df_['src_ip'].map(node_to_index).values
dst = df_['dst_ip'].map(node_to_index).values

Create an Adjacency List The adjacency list is a way to represent the edges of a graph. In this context, it lists which nodes (IP addresses) are connected to which other nodes, thus defining the edges of the graph.

In [ ]:
#Create an Adjacency List
adj = df_[['src_ip', 'dst_ip']].to_numpy()
np.save("adj.npy", adj)


In [ ]:


#binary classification
binary = df_['label'].to_numpy().astype(np.int8)
np.save("label_bi.npy", binary)

In [ ]:
#multi classification
multi = df_['type'].to_numpy().astype(np.int16)
np.save("label_mul.npy", multi)

In [ ]:
dfegfeat=df_.iloc[:,2:37]

In [ ]:
dfegfeat.dtypes

proto                      object
service                    object
duration                  float64
src_bytes                   int64
dst_bytes                   int64
conn_state                 object
missed_bytes                int64
src_pkts                    int64
src_ip_bytes                int64
dst_pkts                    int64
dst_ip_bytes                int64
dns_query                  object
dns_qclass                  int64
dns_qtype                   int64
dns_rcode                   int64
dns_AA                     object
dns_RD                     object
dns_RA                     object
dns_rejected               object
ssl_version                object
ssl_cipher                 object
ssl_resumed                object
ssl_established            object
ssl_subject                object
ssl_issuer                 object
http_trans_depth           object
http_method                object
http_uri                   object
http_version               object
http_request_b

In [ ]:
from sklearn.preprocessing import StandardScaler

liNum = ['duration','src_bytes','dst_bytes','missed_bytes',
         'src_pkts','src_ip_bytes','dst_pkts','dst_ip_bytes','dns_qclass',
         'dns_qtype','dns_rcode','http_request_body_len','http_response_body_len','http_status_code']

sc= StandardScaler()

dfegfeat[liNum]=sc.fit_transform(dfegfeat[liNum])

In [ ]:


edge_feat = dfegfeat.iloc[:,:].to_numpy()
np.save("edge_feat.npy",edge_feat)

In [ ]:
import torch

In [ ]:
from torch_geometric.data import Data

In [ ]:
dfnodefeat=df_.iloc[:,5:40]

In [ ]:
dfnodefeat.dtypes

src_bytes                  int64
dst_bytes                  int64
conn_state                object
missed_bytes               int64
src_pkts                   int64
src_ip_bytes               int64
dst_pkts                   int64
dst_ip_bytes               int64
dns_query                 object
dns_qclass                 int64
dns_qtype                  int64
dns_rcode                  int64
dns_AA                    object
dns_RD                    object
dns_RA                    object
dns_rejected              object
ssl_version               object
ssl_cipher                object
ssl_resumed               object
ssl_established           object
ssl_subject               object
ssl_issuer                object
http_trans_depth          object
http_method               object
http_uri                  object
http_version              object
http_request_body_len      int64
http_response_body_len     int64
http_status_code           int64
http_user_agent           object
http_orig_

In [ ]:
# Identify numeric columns
numeric_cols = dfnodefeat.select_dtypes(include=['int64']).columns

scaler = StandardScaler()

# Apply the scaler to the numeric columns
dfnodefeat[numeric_cols] = scaler.fit_transform(dfnodefeat[numeric_cols])

In [ ]:
# Node features
node_features = dfnodefeat.iloc[:, :].apply(pd.to_numeric, errors='coerce')
  # Adjust the features accordingly'

  # Handle NaN values if necessary (e.g., filling NaNs with zeros or dropping rows with NaNs)
node_features = node_features.dropna()
# Convert to NumPy array
node_features = node_features.to_numpy()

# Convert to PyTorch tensor
x = torch.tensor(node_features, dtype=torch.float)

In [ ]:
# Create edge index
edge_index = torch.tensor([src, dst], dtype=torch.long)

# Create PyTorch Geometric Data object
graph_data = Data(x=x, edge_index=edge_index, y=torch.tensor(multi, dtype=torch.long))

# Display the graph data
print(graph_data)

Data(x=[461043, 35], edge_index=[2, 461043], y=[461043])


<ipython-input-24-21cf2cc99eeb>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  edge_index = torch.tensor([src, dst], dtype=torch.long)


In [ ]:
num_classes = len(np.unique(binary))  # Number of unique classes in the binary labels

In [ ]:
num_classes

2

**GCN  model**

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

# Define a GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(graph_data.num_node_features, 32)
        self.conv2 = GCNConv(32,num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model, optimizer, and loss function
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()


In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate the model
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(graph_data)
        pred = out.argmax(dim=1)
        acc = accuracy_score(graph_data.y.cpu(), pred.cpu())
    return acc
# Training loop
for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        acc = evaluate()
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')

Epoch 0, Loss: 16.455665588378906, Accuracy: 0.3495205436369276
Epoch 10, Loss: 236.81149291992188, Accuracy: 0.5836635628347031
Epoch 20, Loss: 6.046147346496582, Accuracy: 0.6503037677613585
Epoch 30, Loss: 24.32952308654785, Accuracy: 0.48914743310276915
Epoch 40, Loss: 19.56671905517578, Accuracy: 0.6479590840767564
Epoch 50, Loss: 9.779786109924316, Accuracy: 0.6503276267072703
Epoch 60, Loss: 2.1539273262023926, Accuracy: 0.6498005609021285
Epoch 70, Loss: 2.322910785675049, Accuracy: 0.6490891305149411
Epoch 80, Loss: 2.940197467803955, Accuracy: 0.6495988443594198
Epoch 90, Loss: 2.110948324203491, Accuracy: 0.6497181390889787
Epoch 100, Loss: 0.7060307860374451, Accuracy: 0.648180321575211
Epoch 110, Loss: 0.7376192212104797, Accuracy: 0.6466164761204486
Epoch 120, Loss: 0.7119545936584473, Accuracy: 0.6477595365291307
Epoch 130, Loss: 0.6119855642318726, Accuracy: 0.6483668551523394
Epoch 140, Loss: 0.6161268353462219, Accuracy: 0.64781159241112
Epoch 150, Loss: 0.61626416444

**GAT(Gragh Attention Network)**

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

# Determine the number of classes
num_classes = len(np.unique(binary))

# Define a GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_node_features, 32, heads=8, dropout=0.6)
        self.conv2 = GATConv(32 * 8, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
num_node_features = graph_data.num_node_features
modelgat = GAT(num_node_features, num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(modelgat.parameters(), lr=0.05, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
# Train the model
def train():
    modelgat.train()
    optimizer.zero_grad()
    out = modelgat(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate():
    modelgat.eval()
    with torch.no_grad():
        out = modelgat(graph_data)
        pred = out.argmax(dim=1)
        acc = accuracy_score(graph_data.y.cpu(), pred.cpu())
    return acc

# Training loop
for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        acc = evaluate()
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy Gat: {acc}')


Epoch 0, Loss: 1.35775625705719, Accuracy Gat: 0.680099253214993
Epoch 10, Loss: 1.3297932147979736, Accuracy Gat: 0.6798042698837202
Epoch 20, Loss: 1.2984198331832886, Accuracy Gat: 0.6801122671854903
Epoch 30, Loss: 1.2905946969985962, Accuracy Gat: 0.680507024290576
Epoch 40, Loss: 1.2599411010742188, Accuracy Gat: 0.6801708300527283
Epoch 50, Loss: 1.355158805847168, Accuracy Gat: 0.6843353006118735
Epoch 60, Loss: 1.3373111486434937, Accuracy Gat: 0.5532499137824455


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

# Determine the number of classes
num_classes = len(np.unique(binary))

# Define a GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_node_features, 64, heads=8, dropout=0.66)
        self.conv2 = GATConv(64 * 8, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize the model
num_node_features = graph_data.num_node_features
model = GAT(num_node_features, num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.06, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
from sklearn.metrics import accuracy_score

# Train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate the model
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(graph_data)
        pred = out.argmax(dim=1)
        acc = accuracy_score(graph_data.y.cpu(), pred.cpu())
    return acc

# Training loop
for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        acc = evaluate()
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')


Epoch 0, Loss: 299.21832275390625, Accuracy: 0.6507223838123559
Epoch 10, Loss: 270.8002014160156, Accuracy: 0.5102756142051826


KeyboardInterrupt: 